In [4]:
using Plots
plotly()
using ForwardDiff
using MAT  # to load designed pulse


┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /home/jonathan/.julia/packages/Plots/sUjwv/src/backends.jl:372


In [132]:
vars = matread("test_pulses/simulation_testing.mat")

rf_abs = vars["rf_abs"]
rf_phs = vars["rf_phs"]
gamgdt = vars["gamgdt"]
dom0dt = 0
xx = vars["xx"]
b1 = vars["b1"]
plot(transpose(rf_abs))

In [133]:
# Old bloch simulation function (for reference)
function myBlochSim(α)
       
    # α: vector of RF rotations, plus a gradient rotation at the end

    N = length(α) - 4 # number of points in pulse

    
    # initialize magnetization
    Mx = α[end - 2]
    My = α[end - 1]
    Mz = α[end]
    
    # apply prewinding gradient rotation
    #cg = cos(-N / 2 * α[end-3])
    #sg = sin(-N / 2 * α[end-3])
    #Mxi = Mx
    #Myi = My
    #Mx = cg * Mxi + sg * Myi
    #My = -sg * Mxi + cg * Myi
    
    # pre-calculate gradient rotation params
    cg = cos(α[end-3])
    sg = sin(α[end-3])
    for ii = 1 : length(α) - 4
        
        # calculate RF rotation params
        crf = cos(α[ii])
        srf = sin(α[ii])
        
        # apply RF rotation
        Myi = My
        Mzi = Mz
        My = crf * Myi + srf * Mzi
        Mz = -srf * Myi + crf * Mzi
        
        # apply gradient rotation
        Mxi = Mx
        Myi = My
        Mx = cg * Mxi + sg * Myi
        My = -sg * Mxi + cg * Myi
            
    end
    
    # apply rewinding gradient rotation
    #cg = cos(-N / 2 * α[end-3])
    #sg = sin(-N/ 2 * α[end-3])
    #Mxi = Mx
    #Myi = My
    #Mx = cg * Mxi + sg * Myi
    #My = -sg * Mxi + cg * Myi
    
    return Mx, My, Mz
    
end

myBlochSim (generic function with 1 method)

In [134]:
# JBM define new bloch sim function
# rf_mag - abs value of input pulse
# rf_phs - phase of input pulse
# gamgdt - gradient samples in radans/units of xx
# xx - spatial locations
# dom0dt - off-resonance phase in radians
# b1 - b1 at each spatial location, b1*rf should have units of radians

# returns a and b, slr alpha parameters

function abrm_hp(rf_mag,rf_phs, gamgdt, xx, dom0dt=0, b1=None)
    xx = 1 # only care about one spatial location
    Ns = 1 # there is only one spatial location
    Nt = length(rf_mag)
    a = ones((1))
    b = zeros((1))
    for ii = 1 : Nt
        z = exp(1im * gamgdt[ii] + dom0dt)
        b = b*z
        
        b1rf = b1 * rf_abs[:,ii]
        C = cos.(b1rf/2)
        S = 1im * exp.(1im*angle.(b1rf)).*sin.(b1rf/2)  # assuming b1rf > 0

        at = a .* C - b .* conj.(S)
        bt = a .* S + b .* C
        
        a = at
        b = bt
    end
    
    z = exp.(1im/2 * (xx*sum(gamgdt)+Nt*dom0dt))
    a = a * z
    b = b * z
    return a, b
    
end

abrm_hp (generic function with 4 methods)

In [135]:
# test case

a, b = abrm_hp(rf_abs,rf_phs, gamgdt, xx, dom0dt,b1)
print(size(a))

(200,)

In [136]:
Mxy = 2 * conj.(a) .* b

200-element Array{Complex{Float64},1}:
  0.0 + 0.0im
  0.0 + 0.8938387736439212im
  0.0 + 0.8015740535103328im
 -0.0 - 0.1750055990193517im
 -0.0 - 0.9585150495034717im
 -0.0 - 0.684568651098418im
 -0.0 + 0.344609628585057im
 -0.0 + 0.993606639837157im
 -0.0 + 0.5464339526216078im
  0.0 - 0.5035772393526107im
  0.0 - 0.9980304383701758im
  0.0 - 0.39143350262911286im
  0.0 + 0.6470018763588724im
      ⋮
 -0.0 + 0.5242285877662365im
 -0.0 + 0.9962311366054885im
 -0.0 + 0.3691685817776371im
  0.0 - 0.6651692435105914im
  0.0 - 0.9656770601663758im
  0.0 - 0.20082773304023338im
  0.0 + 0.7855793683040274im
  0.0 + 0.9053172418567361im
  0.0 + 0.026288311489633554im
  0.0 - 0.8817424887558175im
  0.0 - 0.8170146949323099im
 -0.0 + 0.1490625020457168im

In [137]:
plot(abs.(Mxy))